# Introduction to data exploration mainly using Pandas

In [ ]:
%matplotlib inline
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. Pandas dataframe and series

The first step will be to open a CSV file which contain some information about some voting count of a referendum in France.

In [ ]:
filename_referendum = os.path.join('data', 'referendum.csv')
filename_referendum

The data are not separated with a comma but a semi-colummn.

In [ ]:
df = pd.read_csv(filename_referendum, sep=';')

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.index

In [ ]:
df.columns

It will be easier with we use the name of the city as an index.

In [ ]:
df = df.set_index('Town name')

In [ ]:
df.head()

## 2. Simple analysis

* What is the city with the most registered people?

In [ ]:
df.loc[:, 'Registered'].head()

In [ ]:
col_registered = df.loc[:, 'Registered']

In [ ]:
col_registered.max()

In [ ]:
col_registered == col_registered.max()

In [ ]:
mask_most_registered = col_registered == col_registered.max()

In [ ]:
col_registered.loc[mask_most_registered]

In [ ]:
df.loc[mask_most_registered]

* What is the city with the least number of registered persons?

In [ ]:
mask_least_registered = col_registered == col_registered.min()

In [ ]:
df.loc[mask_least_registered]

Let's go to the `notebook.ipynb` to formalize the different aspect we just used up to now.

## 3. Group information together.

Let's now make a more advance analysis. Instead to make a micro-analysis by cities, we would like to make a macro-analysis by department.

In [ ]:
df.head()

Therefore, we would like group the votes by department and add them up.

In [ ]:
df.groupby('Department code').sum()

In [ ]:
df_department = df.groupby(['Department code', 'Department name']).sum().reset_index()
df_department

## 4. Merging information together

We would like to plot some of the information into the a map.

In [ ]:
import geopandas as gpd

In [ ]:
gdf_department = gpd.read_file(os.path.join('data', 'departements.geojson'))

In [ ]:
type(gdf_department)

In [ ]:
gdf_department.head()

In [ ]:
gdf_department.plot()

So what we need at that stage is to merge the different dataframe together.

In [ ]:
df = gdf_department.merge(df_department, how='inner', left_on='code', right_on='Department code')

In [ ]:
df.plot(column='Registered')

## 5. Bug correction

In [ ]:
def prepend_zero(code):
    if len(code) == 1:
        return '0' + code
    return code

df_department['Department code'] = df_department['Department code'].apply(prepend_zero)

In [ ]:
df = gdf_department.merge(df_department, how='inner', left_on='code', right_on='Department code')

In [ ]:
df.head()

In [ ]:
df.plot(column='Registered')

In [ ]:
df.plot(column='Choice A')

In [ ]:
df.plot(column='Choice B')

In [ ]:
df_normalized = df.copy()

In [ ]:
df_normalized['Choice A'] /= df[['Choice A', 'Choice B']].sum(axis=1)
df_normalized['Choice B'] /= df[['Choice A', 'Choice B']].sum(axis=1)

In [ ]:
df_normalized.plot(column='Choice B')

In [ ]:
df_normalized.plot(column='Choice A')

## 6. This your turn

The goal will be to repeat the analysis but at the region scale. However, this is not as easy. The regions information in directly available and we will need to import it from another external source.

* Open the `data/referendum.csv` file.

In [ ]:
# %load solutions/24_solutions.py
df_referendum = pd.read_csv(filename_referendum, sep=';')


* Show the 5 first rows.

In [ ]:
# %load solutions/25_solutions.py
df_referendum.head()


As you can see, there is no information about the regions. Before to get to this stage, let's correct the issue with the deparment numbering.

In [ ]:
# %load solutions/26_solutions.py
df_referendum['Department code'] = df_referendum['Department code'].apply(
    lambda x: '0' + x if len(x) == 1 else x
)
df_referendum.head()


* Load the information related to the regions from the file `data/regions.csv`. Show the 5 first rows.

In [ ]:
# %load solutions/27_solutions.py
df_regions = pd.read_csv(os.path.join('data', 'regions.csv'))
df_regions.head()


* Lead the information related to the departments from the file `data/departments.csv`. Show the 5 first rows.

In [ ]:
# %load solutions/28_solutions.py
df_departments = pd.read_csv(os.path.join('data', 'departments.csv'))
df_departments.head()


* Find the column in the departments dataframe which is related to the `code` column of the regions dataframe. Merge both dataframe using these informations. Show the 5 first rows of the resulting dataframe.

In [ ]:
# %load solutions/29_solutions.py
# pd.merge(df_departments, df_regions, how='inner', left_on='region_code', right_on='code')
df_reg_dep = df_departments.merge(df_regions, how='inner', left_on='region_code', right_on='code')
df_reg_dep.head()


* In the previous dataframe as column linked to the department code which could be merged with our referendum data as we did previously. Since we already got information about the regions, we can get the regions dataframe with a new merge. Show the 5 first rows of the merged dataframe.

In [ ]:
# %load solutions/30_solutions.py
df = df_referendum.merge(df_reg_dep, how='inner', left_on='Department code', right_on='code_x')
df.head()


* Group and add up the vote by region and show the resulting dataframe.

In [ ]:
# %load solutions/31_solutions.py
regions_vote = df.groupby(['code_y', 'name_y']).sum()
regions_vote


* Taking example on the previous case, plot the vote for the "choice A" and "choice B". Use the file `regions.geojson` instead of `departments.geojson`.

In [ ]:
# %load solutions/32_solutions.py
gdf_regions = gpd.read_file(os.path.join('data', 'regions.geojson'))
gdf_regions = gdf_regions.merge(regions_vote, how='inner', left_on='code', right_on='code_y')

gdf_normalized = gdf_regions.copy()
gdf_normalized['Choice A'] /= gdf_regions[['Choice A', 'Choice B']].sum(axis=1)
gdf_normalized['Choice B'] /= gdf_regions[['Choice A', 'Choice B']].sum(axis=1)


In [ ]:
# %load solutions/33_solutions.py
gdf_normalized.plot(column='Choice B')


In [ ]:
# %load solutions/34_solutions.py
gdf_normalized.plot(column='Choice A')
